<a href="https://colab.research.google.com/github/Abhiramvp/OPF-with-CVXpy/blob/main/OPF_cvxpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Solving OPF with CVXPY**

**Import libraries**

In [ ]:
!pip -q install pypower scipy cvxpy

import os
import pypower
import numpy as np
import cvxpy as cp

from pypower.loadcase import loadcase
from pypower.ext2int import ext2int
from pypower.makeYbus import makeYbus


**Choose a matpower/pypower case_xx**

In [ ]:
pp_dir = os.path.dirname(pypower.__file__)
case_path = os.path.join(pp_dir, "case9.py")   # change to "case14.py", "case30.py", etc.

ppc = loadcase(case_path)
print("Loaded baseMVA:", ppc["baseMVA"])

ppc = ext2int(ppc)  # internal indexing (recommended)


Loaded baseMVA: 100.0


**Y bus creation**

In [ ]:
baseMVA  = float(ppc["baseMVA"])
bus      = np.array(ppc["bus"], dtype=float)
gen_all  = np.array(ppc.get("gen", []), dtype=float)
branch   = np.array(ppc["branch"], dtype=float)
gencost_all = np.array(ppc.get("gencost", []), dtype=float) if "gencost" in ppc else None

Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)
Ybus = Ybus.tocsr()

nb, nl, ng = bus.shape[0], branch.shape[0], gen_all.shape[0]
print(f"Loaded: {case_path}")
print(f"baseMVA={baseMVA}, nb={nb}, nl={nl}, ng={ng}")
print("Ybus shape:", Ybus.shape, "nnz:", Ybus.nnz)


Loaded: /usr/local/lib/python3.12/dist-packages/pypower/case9.py
baseMVA=100.0, nb=9, nl=9, ng=3
Ybus shape: (9, 9) nnz: 27


**Solve OPF with cvxpy**

In [ ]:
# Bus columns (0-index): type, Pd, Qd, Vm, Vmax, Vmin
BUS_TYPE = bus[:, 1].astype(int)
Pd = bus[:, 2] / baseMVA
Qd = bus[:, 3] / baseMVA
Vm_set = bus[:, 7]
Vmax = bus[:, 11]
Vmin = bus[:, 12]

if gen_all.size == 0:
    raise ValueError("No generators found. SDP-OPF needs at least one generator.")

# Generator filtering (in-service)
gen_status = gen_all[:, 7].astype(int)
keep = (gen_status == 1)
gen = gen_all[keep, :]

gencost = gencost_all[keep, :] if (gencost_all is not None and gencost_all.shape[0] == gen_all.shape[0]) else None

ng = gen.shape[0]
gen_bus = gen[:, 0].astype(int) - 1
Pmax = gen[:, 8] / baseMVA
Pmin = gen[:, 9] / baseMVA
Qmax = gen[:, 3] / baseMVA
Qmin = gen[:, 4] / baseMVA

# Generator-to-bus incidence
Cg = np.zeros((nb, ng))
for k in range(ng):
    Cg[gen_bus[k], k] = 1.0


# SDP variables
W  = cp.Variable((nb, nb), hermitian=True)
Pg = cp.Variable(ng)  # p.u.
Qg = cp.Variable(ng)  # p.u.

constraints = [W >> 0]

# Voltage constraints
for i in range(nb):
    constraints += [cp.imag(W[i, i]) == 0]
    constraints += [cp.real(W[i, i]) >= Vmin[i]**2, cp.real(W[i, i]) <= Vmax[i]**2]
    if BUS_TYPE[i] in (2, 3):  # PV or REF
        constraints += [cp.real(W[i, i]) == Vm_set[i]**2]

# Power injection constraints using Ybus
Y = Ybus.toarray().astype(np.complex128)

def S_inj(i):
    return cp.sum(cp.multiply(np.conj(Y[i, :]), W[i, :]))

P_inj = cp.hstack([cp.real(S_inj(i)) for i in range(nb)])
Q_inj = cp.hstack([cp.imag(S_inj(i)) for i in range(nb)])

constraints += [P_inj == (Cg @ Pg) - Pd]
constraints += [Q_inj == (Cg @ Qg) - Qd]

constraints += [Pg >= Pmin, Pg <= Pmax]
constraints += [Qg >= Qmin, Qg <= Qmax]

# Objective (MATPOWER polynomial gencost model=2); fallback: minimize sum Pg
obj = 0
if gencost is not None and gencost.size > 0:
    for k in range(ng):
        row = gencost[k, :]
        if int(row[0]) != 2:
            obj += Pg[k] * baseMVA
            continue
        ncoef = int(row[3])
        coeffs = row[4:4+ncoef]  # descending
        Pg_MW = Pg[k] * baseMVA
        if ncoef == 3:
            c2, c1, c0 = coeffs
            obj += c2 * cp.square(Pg_MW) + c1 * Pg_MW + c0
        elif ncoef == 2:
            c1, c0 = coeffs
            obj += c1 * Pg_MW + c0
        else:
            obj += Pg_MW
else:
    obj = cp.sum(Pg)

prob = cp.Problem(cp.Minimize(obj), constraints)


**Display Results**

In [ ]:
prob.solve(solver=cp.SCS, verbose=False)

print("=== SDP OPF (Relaxation) ===")
print("status:", prob.status)
print("objective:", prob.value)

print("Pg (MW):", Pg.value * baseMVA)
print("Qg (MVAr):", Qg.value * baseMVA)

Wval = W.value
Vmag = np.sqrt(np.maximum(np.real(np.diag(Wval)), 0))
evals = np.linalg.eigvalsh(Wval)

print("Vmag (pu):", Vmag)
print("W eigenvalues:", evals)
print("approx rank (eigs > 1e-6):", int(np.sum(evals > 1e-6)))


=== SDP OPF (Relaxation) ===
status: optimal
objective: 5277.575274382554
Pg (MW): [ 87.66946497 134.46867762  95.40678718]
Qg (MVAr): [39.11821174 -9.90845676 -7.62415945]
Vmag (pu): [1.         1.         1.         1.00870378 0.9914496  1.00284577
 0.9912453  1.00656552 1.01283591]
W eigenvalues: [-3.15586573e-05 -1.49894005e-05 -9.01439723e-06 -3.63977036e-07
  8.14243009e-06  2.38636686e-05  4.66755924e-05  5.04951812e-03
  9.02266097e+00]
approx rank (eigs > 1e-6): 5
